In [1]:
# Librerías.
import pandas as pd
import Funciones as f
import unidecode

In [ ]:
%%capture
%run "10. Redes sociales.ipynb"

In [3]:
# Lista de medios de prensa.
Medios_Prensa = [
    'ambito financiero',
    'prensa obrera',
    'diario universal',
    'popular',
    'izquierda diario',
    'clarin',
    'perfil',
    'pagina 12',
    'infobae',
    'el cronista',
    'la nacion',
    'tiempo argentino',
    'ninguno'
]

In [4]:
# Eliminar acentos y convertir a minúsculas en la columna Medios_Prensa.
for Nombre_df, df in dfs_Finales.items():
   df['Medios_Prensa'] = df['Medios_Prensa'].apply(lambda x: unidecode.unidecode(str(x)).lower() if pd.notna(x) else x)

In [5]:
# Crear una columna para cada red social.

for Nombre_df, df in dfs_Finales.items():
   for Medio in Medios_Prensa:
       
       # Crear la columna con nombre limpio.
       Nombre_Columna = f"{Medio}"
       
       # Verificar si el medio está en el string de cada fila.
       df[Nombre_Columna] = df['Medios_Prensa'].str.contains(Medio, case = False, na = False).astype(int)

In [6]:
# Crear diccionario de mapeo para renombrar columnas a Pascal_Snake_Case.
Mapeo_Medios_Prensa = {Medio: Medio.title().replace(' ', '_') for Medio in Medios_Prensa}

# Aplicar el renombrado a todos los DataFrames.
for Nombre_df, df in dfs_Finales.items():
  # Verificar qué columnas existen en este DataFrame.
  Columnas_Existentes = [Columna for Columna in Medios_Prensa if Columna in df.columns]
  
  if Columnas_Existentes:       
      # Crear mapeo solo para las columnas que existen.
      Mapeo_Actual = {Columna: Columna.title().replace(' ', '_') for Columna in Columnas_Existentes}
      
      # Renombrar las columnas.
      df.rename(columns = Mapeo_Actual, inplace=True)

In [7]:
# Renombrar columna "Ninguno".
for Nombre_df, df in dfs_Finales.items():
      df.rename(columns = {'Ninguno': 'Medios_Prensa_Ninguno'}, inplace=True)

In [ ]:
print("="*80)
print("INVESTIGACIÓN PROFUNDA: DIFERENCIA EN COHERENCIA DE MEDIOS")
print("="*80)

for Nombre_df, df in dfs_Finales.items():
    print(f"\n🔍 DATAFRAME: {Nombre_df}")
    print("-" * 50)
    
    # Recrear las columnas necesarias para el análisis.
    if 'Medios_Prensa' in globals() and 'Medios_Prensa' in df.columns:
        
        # Identificar columnas de medios reales (excluyendo "Ninguno").
        Medios_Pascal_Snake = [Medio.title().replace(' ', '_') for Medio in Medios_Prensa if Medio != 'ninguno']
        Columnas_Medios_Reales = [col for col in Medios_Pascal_Snake if col in df.columns]
        
        if len(Columnas_Medios_Reales) > 0:
            # Recalcular para verificar.
            df['Total_Medios_Temp'] = df[Columnas_Medios_Reales].sum(axis=1)
            Sin_Medios_Calculado = (df['Total_Medios_Temp'] == 0).sum()
            
            if 'Medios_Prensa_Ninguno' in df.columns:
                Ninguno_Declarado = df['Medios_Prensa_Ninguno'].sum()
                print(f"   Sin medios (calculado): {Sin_Medios_Calculado}")
                print(f"   'Ninguno' declarado: {Ninguno_Declarado}")
                Diferencia = abs(Sin_Medios_Calculado - Ninguno_Declarado)
                print(f"   Diferencia: {Diferencia} casos")
                
                if Diferencia > 0:
                    print(f"\n   🔍 ANALIZANDO LOS {Diferencia} CASOS PROBLEMÁTICOS:")
                    
                    # 1. Casos sin medios detectados pero que NO marcaron "Ninguno".
                    Casos_Sin_Medios_Sin_Ninguno = df[
                        (df['Total_Medios_Temp'] == 0) & 
                        (df['Medios_Prensa_Ninguno'] == 0)
                    ]
                    
                    print(f"\n   A) CASOS SIN MEDIOS DETECTADOS QUE NO MARCARON 'NINGUNO': {len(Casos_Sin_Medios_Sin_Ninguno)}")
                    
                    if len(Casos_Sin_Medios_Sin_Ninguno) > 0:
                        print("   " + "="*60)
                        print("   ID    | Medios_Prensa Original")
                        print("   " + "-"*60)
                        
                        for idx, (index, fila) in enumerate(Casos_Sin_Medios_Sin_Ninguno.head(10).iterrows()):
                            ID = fila['ID']
                            Medios_Original = fila['Medios_Prensa']
                            print(f"   {ID:<5} | {str(Medios_Original)[:50]}")
                        
                        if len(Casos_Sin_Medios_Sin_Ninguno) > 10:
                            print(f"   ... y {len(Casos_Sin_Medios_Sin_Ninguno) - 10} casos más")
                        
                        # Analizar patrones en estos casos.
                        print(f"\n   📊 ANÁLISIS DE PATRONES:")
                        
                        # Valores únicos en Medios_Prensa para estos casos.
                        Valores_Problematicos = Casos_Sin_Medios_Sin_Ninguno['Medios_Prensa'].value_counts()
                        print(f"   Valores más frecuentes en Medios_Prensa:")
                        for Valor, Frecuencia in Valores_Problematicos.head(10).items():
                            print(f"     '{Valor}': {Frecuencia} casos")
                        
                        # Buscar medios no contemplados en la lista.
                        print(f"\n   🔍 MEDIOS NO CONTEMPLADOS EN LA LISTA:")
                        Medios_No_Contemplados = set()
                        
                        for Medios_Texto in Casos_Sin_Medios_Sin_Ninguno['Medios_Prensa'].dropna():
                            if isinstance(Medios_Texto, str) and len(Medios_Texto.strip()) > 0:
                                # Dividir por comas y analizar cada parte.
                                Partes = [parte.strip().lower() for parte in Medios_Texto.split(',')]
                                for Parte in Partes:
                                    if len(Parte) > 2:  # Ignorar texto muy corto.
                                        # Verificar si esta parte no está en nuestra lista.
                                        Es_Conocido = False
                                        for Medio_Lista in Medios_Prensa:
                                            if Medio_Lista.lower() in Parte or Parte in Medio_Lista.lower():
                                                Es_Conocido = True
                                                break
                                        
                                        if not Es_Conocido:
                                            Medios_No_Contemplados.add(Parte)
                        
                        if Medios_No_Contemplados:
                            print(f"   Medios potenciales no incluidos en la lista:")
                            for Medio in sorted(list(Medios_No_Contemplados)[:15]):
                                Frecuencia = sum(1 for texto in Casos_Sin_Medios_Sin_Ninguno['Medios_Prensa'].dropna() 
                                               if isinstance(texto, str) and Medio in texto.lower())
                                print(f"     '{Medio}': {Frecuencia} menciones")
                        else:
                            print(f"   No se encontraron medios claramente identificables")
                    
                    # 2. Casos que marcaron "Ninguno" pero tienen medios detectados.
                    Casos_Ninguno_Con_Medios = df[
                        (df['Total_Medios_Temp'] > 0) & 
                        (df['Medios_Prensa_Ninguno'] == 1)
                    ]
                    
                    print(f"\n   B) CASOS QUE MARCARON 'NINGUNO' PERO TIENEN MEDIOS: {len(Casos_Ninguno_Con_Medios)}")
                    
                    if len(Casos_Ninguno_Con_Medios) > 0:
                        print("   " + "="*70)
                        print("   ID    | Medios_Prensa Original              | Medios Detectados")
                        print("   " + "-"*70)
                        
                        for idx, (index, fila) in enumerate(Casos_Ninguno_Con_Medios.head(5).iterrows()):
                            ID = fila['ID']
                            Medios_Original = fila['Medios_Prensa']
                            
                            # Listar medios detectados.
                            Medios_Detectados = []
                            for Col in Columnas_Medios_Reales:
                                if fila[Col] == 1:
                                    Medios_Detectados.append(Col)
                            
                            Medios_Str = ", ".join(Medios_Detectados[:3])
                            if len(Medios_Detectados) > 3:
                                Medios_Str += f" (+{len(Medios_Detectados)-3} más)"
                            
                            print(f"   {ID:<5} | {str(Medios_Original)[:35]:<35} | {Medios_Str}")
                    
                    # 3. Análisis detallado de casos específicos.
                    print(f"\n   🔬 ANÁLISIS DETALLADO DE CASOS ESPECÍFICOS:")
                    
                    # Tomar 3 casos representativos.
                    if len(Casos_Sin_Medios_Sin_Ninguno) > 0:
                        print(f"\n   Análisis detallado de 3 casos sin medios:")
                        
                        for i, (index, fila) in enumerate(Casos_Sin_Medios_Sin_Ninguno.head(3).iterrows()):
                            print(f"\n   CASO {i+1} - ID: {fila['ID']}")
                            print(f"   Medios_Prensa: '{fila['Medios_Prensa']}'")
                            
                            # Verificar detección manual.
                            if pd.notna(fila['Medios_Prensa']):
                                Texto_Medios = str(fila['Medios_Prensa']).lower()
                                print(f"   Texto normalizado: '{Texto_Medios}'")
                                
                                print("   Verificación manual de detección:")
                                for Medio_Original in Medios_Prensa[:8]:  # Primeros 8.
                                    Deberia_Detectar = Medio_Original in Texto_Medios
                                    Columna_Pascal = Medio_Original.title().replace(' ', '_')
                                    if Columna_Pascal == 'Ninguno':
                                        Columna_Pascal = 'Medios_Prensa_Ninguno'
                                    
                                    Detectado = fila.get(Columna_Pascal, 0) if Columna_Pascal in df.columns else 0
                                    
                                    Status = "✅" if (Detectado == 1) == Deberia_Detectar else "❌"
                                    print(f"     {Medio_Original:<18}: Esperado={int(Deberia_Detectar)}, Real={Detectado} {Status}")
                                
                                # Buscar posibles matches parciales.
                                print("   Posibles matches parciales:")
                                for Medio_Original in Medios_Prensa:
                                    if len(Medio_Original) > 4:  # Solo medios con nombres largos.
                                        Palabras_Medio = Medio_Original.split()
                                        for Palabra in Palabras_Medio:
                                            if len(Palabra) > 3 and Palabra in Texto_Medios:
                                                print(f"     Encontrada palabra '{Palabra}' de '{Medio_Original}'")
                    
                    # 4. Recomendaciones para corrección.
                    print(f"\n   💡 RECOMENDACIONES PARA CORRECCIÓN:")
                    
                    if len(Medios_No_Contemplados) > 0:
                        print("   1. AGREGAR MEDIOS FALTANTES A LA LISTA:")
                        Medios_Sugeridos = sorted(list(Medios_No_Contemplados)[:10])
                        for Medio in Medios_Sugeridos:
                            if len(Medio) > 3 and len(Medio) < 30:  # Filtrar por longitud razonable.
                                print(f"      '{Medio}'")
                    
                    print("   2. MEJORAR DETECCIÓN DE VARIANTES:")
                    Variantes_Sugeridas = {
                        'clarin': ['diario clarin', 'clarín'],
                        'la nacion': ['diario la nacion', 'lanacion'],
                        'pagina 12': ['página 12', 'pagina12'],
                        'infobae': ['infobae.com']
                    }
                    
                    for Medio_Base, Variantes in Variantes_Sugeridas.items():
                        print(f"      {Medio_Base}: {', '.join(Variantes)}")
                    
                    print("   3. REVISAR MANUALMENTE CASOS AMBIGUOS:")
                    print("      - Verificar textos con múltiples comas")
                    print("      - Revisar respuestas en blanco vs 'ninguno'")
                    print("      - Confirmar casos donde aparece 'ninguno' con otros medios")
                
                else:
                    print("   ✅ No hay diferencias - coherencia perfecta")
            
            # Limpiar columna temporal.
            if 'Total_Medios_Temp' in df.columns:
                df.drop('Total_Medios_Temp', axis=1, inplace=True)

print("\n" + "="*80)
print("✅ INVESTIGACIÓN DE DIFERENCIA COMPLETADA")
print("="*80)

# Código adicional para generar lista mejorada de medios.
print("\n" + "="*80)
print("CÓDIGO SUGERIDO PARA MEJORAR LA DETECCIÓN")
print("="*80)

print("""
# Lista mejorada de medios (agregar al notebook original):
Medios_Prensa_Mejorada = [
    'ambito financiero', 'ambito', 'ámbito financiero',
    'prensa obrera',
    'diario universal',
    'popular', 'diario popular',
    'izquierda diario',
    'clarin', 'clarín', 'diario clarin',
    'perfil', 'revista perfil',
    'pagina 12', 'página 12', 'pagina12',
    'infobae', 'infobae.com',
    'el cronista', 'cronista',
    'la nacion', 'lanacion', 'diario la nacion',
    'tiempo argentino',
    'ninguno', 'no leo', 'no consumo'
]

# También considerar agregar estos medios encontrados:
# [Aquí se mostrarían los medios no contemplados detectados]
""")

print("\n📝 NOTA: Ejecuta este código para identificar exactamente")
print("qué medios faltan y cómo mejorar la detección.")
print("="*80)

In [9]:
# Exportar base para el control.
for Nombre_df, df in dfs_Finales.items():
    dfs_Finales[Nombre_df].head(50).to_excel(f'Controles/11. Medios ({Nombre_df}).xlsx', index=False)